# Analiza sentymentu z wykorzystaniem BERT i zbioru Amazon Reviews

## Czynności przygotowawcze
Parametry:

In [58]:
from ReviewDataset import create_data_loader
from SentimentClassifier import SentimentClassifier
from training import train_epoch, eval_model, show_confusion_matrix, get_predictions

datasetUrl = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz'
dataPath = './data/reviews.json.gz'

Dane karty graficznej:

In [59]:
!nvidia-smi

Sat May  1 17:31:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.40       Driver Version: 461.40       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:26:00.0  On |                  N/A |
| 28%   33C    P8    21W / 120W |   1049MiB /  6144MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Importowane biblioteki:

In [60]:
%load_ext autoreload
%autoreload 2
import os
import transformers
import pandas as pd
import gzip
import wget
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from imblearn.under_sampling import RandomUnderSampler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Pobieranie zbioru danych i rozpakowywanie:

In [61]:
try:
    os.stat('./data/')
except:
    os.mkdir('./data/')

try:
    f = open(dataPath)
except IOError:
    print("File not accessible, downloading")
    wget.download(datasetUrl, out=dataPath)
finally:
    if 'f' in locals():
        f.close()

Ładowanie danych:

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(dataPath)

## Analiza danych:

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

Histogram ilości słów w opinii:

In [ ]:
sns.countplot(x='overall', data=df)
plt.xlabel('review score')

In [ ]:
numbers_of_words = df['reviewText'].str.split().str.len()
plot = sns.displot(numbers_of_words, )
plot.set_axis_labels('Length of review in words', 'Count')

Jak widać zbiór jest mocno niezbalansowany, więc istnieje konieczność jego zbalansowania. Ze względu na tekstowy charakter zbioru zastosowanie generującego upsamplingu odpada, iilość danych w zbiorze jest duża, więc undersampling będzie odpowiednim rozwiązaniem.

Przeważająca większość opinii jest krótsza niż 512 słów.

In [ ]:
class_names = ['1', '2', '3', '4', '5']
RANDOM_SEED = 1232
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED, stratify=df[['overall']])
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, stratify=df_test[['overall']])


undersample = RandomUnderSampler(sampling_strategy='not minority')
df_train_undersample, _ = undersample.fit_resample(df_train, df_train[['overall']])

Podział klas zbioru treningowego:

In [ ]:
sns.countplot(x='overall',data=df_train_undersample)
plt.xlabel('review score')

Zbiór zostanie podzielony na zbiór testowy i trenujący z zachowaniem propocji klas, hiperparametry modelu będą wyznaczane w procesie strojenia na zbiorze treningowym z wykorzystaniem k-krotnej walidacji krzyżowej. Podczas jednej iteracji k-krotnej walidacji krzyżowej zbiór trenujący będzie poddawany downsamplingowi w celu zbalansowania klas, a zbiór testujący pozostanie bez zmian.

Ogólnie muszę zrobić takie pipeliny jak na zumach. O ile w ogóle jest potrzebne strojenie hiperparametrów, może wystarczy po prostu k-krotna bez wstępnego train test splita.

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Jak wynika z analizy, większość słów mieści się w zakresie 0-512 słów, więc ustalenie limitu tokenów BERT na górny próg - 512 będzie wystarczające. Dłuższe opinie zostaną skrócone do długości 512 tokenów.

Przygotowanie do treningu:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TOKEN_MAX_LEN = 512
BATCH_SIZE = 16
EPOCHS = 10

train_data_loader = create_data_loader(df_train_undersample, tokenizer, TOKEN_MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, TOKEN_MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, TOKEN_MAX_LEN, BATCH_SIZE)

model = SentimentClassifier(len(class_names))
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

Trening:

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

## Wyniki treningu

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

## Ewaluacja na zbiorze testowym:

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

Macierz błędu:

In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

In [ ]:
review_text = "I love completing my todos! Best app ever!!!"
encoded_review = tokenizer.encode_plus(
  review_text,
  max_length=TOKEN_MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')